In [22]:
import os
import pickle
import json
import random
import time
import datasets

fin = open('/home/mcwave/code/automath/atp/datasets/prover_training_data_6.json', "r")
data = json.load(fin)
print("Data loaded")
data_2 = []

#Load input data
for example in data:
    example[-1] = example[-1].split("\n")[0]
    data_2.append(tuple(example))

data = data_2
random.shuffle(data)
print(len(data), "examples loaded")

Data loaded
4702639 examples loaded


In [6]:
from transformers import AutoTokenizer

tokenizer_name = "morph-labs/morph-prover-v0-7b" #"internlm/internlm2-math-7b" #"ScalableMath/Lean-STaR-plus"  # 'Saisam/gpt-neo-math-small' #

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
tokenizer.padding_side = "right"

# Define the separator token
sep_token = "<sep>"
pad_token = "<pad>"
eos_token = "<end>"

# Check if the separator token already exists in the vocabulary
if sep_token not in tokenizer.get_vocab():
    tokenizer.add_tokens([sep_token])
if pad_token not in tokenizer.get_vocab():
    tokenizer.add_tokens([pad_token])

# Set the separator token
tokenizer.sep_token = sep_token
tokenizer.pad_token = pad_token

tokenizer.add_special_tokens({
    'sep_token': sep_token,
    'pad_token': pad_token,
    'eos_token': eos_token
})
# tokenizer = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-tacgen-byt5-small")
# tokenizer.sep_token = tokenizer.pad_token

3

In [3]:
# state_pps = []
# tacs = []

# #Tokenize input data
# start_time = time.time()
# data_len = len(data)
# for i in range(100000):
#     state_pp = data[i][0]
#     tac = data[i][1]
#     tokens = tokenizer.encode(state_pp)
#     if len(tokens) < 256:
# #         print(state_pp)
# #         print(tokenizer.decode(tokens[1:]))
# #         break
#         if tokenizer.decode(tokens[1:]) == state_pp: 
#             state_pps.append(state_pp)
#             tacs.append(tac)
#     if i % 10000 == 0:
#         print(i, time.time() - start_time)

0 0.0005257129669189453
10000 17.104671955108643
20000 33.486328125
30000 50.46138286590576
40000 66.78817486763
50000 83.59446382522583
60000 100.31472396850586
70000 116.90418672561646
80000 133.41736149787903
90000 150.0275523662567


In [44]:
# fin = open('/home/mcwave/code/automath/atp/ReProver/data/leandojo_benchmark_4/novel_premises/train.json', "r")
# data = json.load(fin)
# state_pps = []
# tacs = []

# for ex in data:
#     try:
#         state_info = ex["traced_tactics"]
#         for tac in state_info:
#             state_pps.append(tac["state_before"])
#             tacs.append(tac["tactic"])
#     except:
#         pass

In [48]:
import torch
import os
import pickle
import json
import random
import time
import datasets
from datasets import Dataset

MAX_LENGTH = 256

# Assuming you have two lists: instructions and responses

instructions = state_pps
responses = tacs

# Function to tokenize and prepare the data
def prepare_data(examples):
    # Concatenate instruction and response with a separator
    full_texts = [f"{instruction} <sep> {response}" for instruction, response in zip(examples['instruction'], examples['response'])]
#     print(full_texts[0])
#     print(1/0)
    # Tokenize the full texts
    encodings = tokenizer(full_texts, truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors='pt')
    #print(encodings)
    
    # Create attention masks: 1 for response tokens, 0 for instruction tokens and padding
    attention_masks = []
    labels = []
    it = 0

    for input_ids in encodings['input_ids']:
        attention_mask = torch.zeros_like(input_ids)
        label = input_ids.clone()
        
        pad_token_idx = (input_ids == tokenizer.pad_token_id).nonzero()
        end_idx = pad_token_idx[0].item() if len(pad_token_idx) > 0 else len(input_ids)
        sep_token_idx = (input_ids == tokenizer.sep_token_id).nonzero()
        #print("sep_token_idx:", sep_token_idx)
        if len(sep_token_idx) == 0:
            sep_token_idx = 0
        else:
            sep_token_idx = sep_token_idx.item()

        attention_mask[0:end_idx+1] = 1
        attention_masks.append(attention_mask)
        
        label[0:sep_token_idx] = -100
        labels.append(label)
#         print(input_ids)
    
    return {
        'input_ids': encodings['input_ids'],
        'attention_mask': torch.stack(attention_masks),
        'labels': torch.stack(labels)
    }

# Create the Hugging Face dataset
dataset = Dataset.from_dict({
    'instruction': instructions,
    'response': responses
})

# Apply the tokenization and preparation function
tokenized_dataset = dataset.map(
    prepare_data,
    batched=True
    #remove_columns=dataset.column_names
)

Map:   0%|          | 0/246714 [00:00<?, ? examples/s]

In [53]:
# def add_end_token(example):
#     end_idx = 0
#     while end_idx < len(example["attention_mask"]) - 1 and example["attention_mask"][end_idx] == 1:
#         end_idx += 1
#     example["input_ids"][end_idx-1] = tokenizer.eos_token_id
#     example["labels"][end_idx-1] = tokenizer.eos_token_id
#     for i in range(len(example["input_ids"])):
# #         print(example["input_ids"][i])
#         if example["input_ids"][i] == tokenizer.sep_token_id:
#             example["labels"][i] = tokenizer.sep_token_id
#             break
# #     print(example["input_ids"])
# #     print(1/0)
#     return example

# tokenized_dataset_2 = tokenized_dataset.map(add_end_token)

# tokenized_dataset_2.save_to_disk("datasets/reprover_dataset.dataset")

tokenized_dataset = datasets.load_from_disk("datasets/reprover_dataset.dataset")
tokenized_dataset = tokenized_dataset_2.train_test_split(train_size=0.95)
tokenized_train_dataset = tokenized_dataset["train"]
tokenized_test_dataset = tokenized_dataset["test"]

Map:   0%|          | 0/246714 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/246714 [00:00<?, ? examples/s]

KeyError: "Column train not in the dataset. Current columns in the dataset: ['instruction', 'response', 'input_ids', 'attention_mask', 'labels']"

In [17]:
test_data = tokenized_train_dataset[40]
for i in range(512):
    print(test_data["input_ids"][i], test_data["attention_mask"][i], test_data["labels"][i])

1 1 -100
264 1 -100
287 1 -100
714 1 -100
12082 1 -100
13 1 -100
2799 1 -100
714 1 -100
264 1 -100
13 1 -100
28716 1 -100
28726 1 -100
714 1 -100
28705 1 -100
29873 1 -100
28726 1 -100
13 1 -100
28716 1 -100
28721 1 -100
28722 1 -100
714 1 -100
12082 1 -100
13 1 -100
229 1 -100
141 1 -100
165 1 -100
28705 1 -100
29873 1 -100
28726 1 -100
28705 1 -100
32000 1 32000
28705 1 28705
1290 1 1290
28720 1 28720
733 1 733
28716 1 28716
28726 1 28726
28793 1 28793
32002 1 32002
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 32001
32001 0 3

IndexError: list index out of range

In [ ]:
# tokenized_dataset = datasets.load_from_disk("datasets/atp_dataset_0821.dataset").train_test_split(train_size=0.95)
# tokenized_train_dataset = tokenized_dataset["train"]
# tokenized_test_dataset = tokenized_dataset["test"]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch
import torch.nn.init as init
from transformers import AutoModel

def initialize_weights(model, init_type='xavier'):
    for module in model.modules():
        if isinstance(module, torch.nn.Linear):
            if init_type == 'xavier':
                init.xavier_uniform_(module.weight)
            elif init_type == 'he':
                init.kaiming_uniform_(module.weight, nonlinearity='relu')
            if module.bias is not None:
                module.bias.data.fill_(0)

model = AutoModelForCausalLM.from_pretrained(
    'models/trained_model_gptneo350m-2/checkpoint-100000',
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
model.resize_token_embeddings(len(tokenizer))
                
#model = AutoModelForCausalLM.from_pretrained("google/gemma-2b").to("cuda")
# model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
# model = AutoModelForCausalLM.from_pretrained("data/results-gptneo350m-2/checkpoint-545000").to("cuda") #"EleutherAI/gpt-neo-1.3B")

In [ ]:
from transformers import Trainer, TrainingArguments
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="models/trained_model_gptneo350m-2",
    evaluation_strategy="steps", #"epochs"
    learning_rate=2e-5,  # PAY ATTENTION TO LEARNING RATE!
    weight_decay=0.01,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=1,
    num_train_epochs=4,
    bf16=True,
    max_grad_norm=1.0,
    save_steps=20000,
    eval_steps=20000,
    logging_steps=1000,
    save_total_limit=3,
    #load_best_model_at_end=True,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=data_collator,
)

cp_path = 'models/trained_model_gptneo350m-2/checkpoint-100000'

trainer.train(cp_path)

In [ ]:
trainer.save_model("models/prover-gemma2-v2/checkpoint-350000")

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW
from tqdm import tqdm
import os

class InstructionResponseDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        instruction, response = self.data[idx]
        
        input_encoding = self.tokenizer(instruction, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        output_encoding = self.tokenizer(response, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")

        input_mask = input_encoding["attention_mask"].squeeze()

        return {
            "input_ids": input_encoding["input_ids"].squeeze(),
            "attention_mask": input_mask,
            "labels": output_encoding["input_ids"].squeeze()
        }

def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Initialize tokenizer and model
model_name = "google/byt5-base"  # You can change this to any other Seq2Seq model
model_name = "./models/trained_model_base_256k"
tokenizer = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-tacgen-byt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# # Prepare your data
# data = [
#     ("Write a poem about spring.", "Blossoms bloom in gentle breeze,\nSunshine warms the earth with ease.\nBirds return with joyful song,\nDays grow longer, winter's gone."),
#     ("Explain photosynthesis.", "Photosynthesis is the process by which plants use sunlight, water, and carbon dioxide to produce oxygen and energy in the form of sugar."),
#     ("What is the capital of France?", "The capital of France is Paris."),
#     ("Describe the water cycle.", "The water cycle involves evaporation, condensation, precipitation, and collection of water on Earth's surface and in the atmosphere."),
#     # Add more instruction-response pairs here
# ]

# Create full dataset
full_dataset = InstructionResponseDataset(data, tokenizer)

# Split into train and test sets
train_size = int(0.9 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=6, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=6, shuffle=False)

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# for i in range(100):
#     print(1,i,train_dataset[i]["input_ids"].shape)
#     print(2,i,train_dataset[i]["attention_mask"].shape)
#     print(3,i,train_dataset[i]["labels"].shape)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Save the model after each epoch
    save_path = f"./models/trained_model_epoch_{epoch + 1}"
    os.makedirs(save_path, exist_ok=True)
    model.save_pretrained(save_path)
    tozzkenizer.save_pretrained(save_path)
    
    avg_train_loss = total_loss / len(train_dataloader)
    avg_test_loss = evaluate(model, test_dataloader, device)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Average Train Loss: {avg_train_loss:.4f}")
    print(f"Average Test Loss: {avg_test_loss:.4f}")

In [ ]:
model.save_pretrained(f"./models/trained_model_base_343k")

In [ ]:
# Test the model
def generate_response(instruction):
    input_ids = tokenizer(instruction, return_tensors="pt").input_ids.to(device)
    print(input_ids)
    output = model.generate(input_ids, max_length=200)
    return tokenizer.decode(output[0], skip_special_tokens=True)


test_instruction = """a b c : ℝ
⊢ a * (c * b) = b * (a * c)
"""

response = generate_response(test_instruction)
print(f"Instruction: {test_instruction}")
print(f"Response: {response}")